# <span style="color:green"> Petits pièges en Python
### Variables globales et locales

En Python, les variables globales sont accessibles dans les fonctions :






In [1]:
x = 10

def my_function():
    print(x)

my_function()

10


Mais quand je modifie : 

In [2]:
x = 10

def my_function():
    x = x + 20
    print(x)
    
my_function()
print(x)

UnboundLocalError: cannot access local variable 'x' where it is not associated with a value

Pour pouvoir modifier la variable globale, il faut utiliser le mot clé `global` :

In [3]:
x = 10

def my_function():
    global x
    x = x + 20
    print(x)
    
my_function()
print(x)


30
30


Maintenant, si j'avais une fonction imbriquée, comment accéder à une variable de la fonction mère ?


In [4]:
def outer_function():
    x = 10
    def inner_function():
        print(x)
    inner_function()
    
outer_function()

del x

10


Mais si je souhaitais modifier la variable de la fonction mère ? 

In [5]:
def outer_function():
    x = 10
    def inner_function():
        global x
        x = x + 20
        print(x)
    inner_function()
    
outer_function()

NameError: name 'x' is not defined

In [6]:
def outer_function():
    x = 10
    def inner_function():
        nonlocal x
        x = x + 20
        print(x)
    inner_function()
    
outer_function()

30


In [7]:
def outer_function():
    x = 10
    
    def intermediate_function():
        def inner_function():
            nonlocal x
            x = x + 20
            print(x)
        inner_function()
    intermediate_function()
    
outer_function()

30


### Les paramètres par défaut

Les paramètres par défaut sont évalués une seule fois lors de la définition de la fonction. Cela peut entraîner des problèmes si le paramètre par défaut est mutable. Par exemple :

In [8]:
def operation(element, mutable=[]):
    mutable.append(element)
    return mutable

my_list = operation(12)

print(my_list)
my_other_list = operation(42)

print(my_other_list)


[12]
[12, 42]


### Late binding closures

Les fonctions imbriquées peuvent accéder aux variables de la fonction parente. Cependant, si ces variables sont modifiées, cela peut entraîner des problèmes. Par exemple :

In [9]:
def create_multipliers():
    return [lambda x : i * x for i in range(5)]

for multiplier in create_multipliers():
    print(multiplier(2))

8
8
8
8
8


# <span style="color:green">Garbage Collection (Ramasse-miettes) </span>

Lorsqu'un objet n'est plus utilisé, il doit être détruit pour libérer la mémoire. Dans certains langages, cela doit être fait manuellement. En Python, le ramasse-miettes (garbage collector) s'en charge automatiquement.

Le ramasse-miettes est un processus qui recherche les objets qui ne sont plus utilisés par le programme et les détruit. Il est activé automatiquement et ne nécessite pas d'intervention de la part du programmeur.

Pour des cas d'usages spécifiques, il est possible de désactiver le ramasse-miettes. Cela peut être utile pour des cas d'usages spécifiques, mais cela peut aussi entraîner des fuites de mémoire.

CPython, par exemple, fait sa garbage collection en utilisant un algorithme de reference counting. Lorsqu'un objet est créé, Python lui rajoute une partie PyObject qui contient son type et un compteur de références. 
<pre>
    object -----> +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+ \
                  |                    ob_refcnt                  | |
                  +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+ | PyObject_HEAD
                  |                    *ob_type                   | |
                  +--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+--+ /
                  |                      ...                      |
</pre>

Lorsque le compteur de références atteint zéro, l'objet sera détruit à la prochaine passe du garbage collector. Pour les cas plus complexes, Python utilise un algorithme de marquage et de balayage.

Pour optimiser la performance, la garbage collection maintient trois générations d'objets :
- La première génération contient les objets qui viennent d'être créés.
- La deuxième génération contient les objets qui ont survécu à une première passe de garbage collection.
- La troisième génération contient les objets qui ont survécu à plusieurs passes de garbage collection.


Pour contrôler le ramasse-miettes, il est possible d'utiliser le module `gc` de Python.

In [10]:
import gc 
import sys

x = object()
print("nombre de références: ", sys.getrefcount(x))


y = x
print("nombre de références: ", sys.getrefcount(x))
print("liste des références: ", gc.get_referrers(x))



nombre de références:  2
nombre de références:  3
liste des références:  [{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'x = 10\n\ndef my_function():\n    print(x)\n\nmy_function()', 'x = 10\n\ndef my_function():\n    x = x + 20\n    print(x)\n    \nmy_function()\nprint(x)', 'x = 10\n\ndef my_function():\n    global x\n    x = x + 20\n    print(x)\n    \nmy_function()\nprint(x)', 'def outer_function():\n    x = 10\n    def inner_function():\n        print(x)\n    inner_function()\n    \nouter_function()\n\ndel x', 'def outer_function():\n    x = 10\n    def inner_function():\n        global x\n        x = x + 20\n        print(x)\n    inner_function()\n    \nouter_function()', 'def outer_function():\n    x = 10\n    def inner_function():\n        nonlocal x\n       

In [11]:
import time 

print(gc.get_threshold())

# test for the default values
start = time.perf_counter()
for a in range(1_000_000):
    x = object()

print(f"for threshold {gc.get_threshold()} time: {time.perf_counter() - start}")

# test for larger values
gc.set_threshold(10000, 1000, 100)
start = time.perf_counter()

for a in range(1_000_000):
    x = object()

print(f"for threshold {gc.get_threshold()} time: {time.perf_counter() - start}")


# back to default

gc.set_threshold(700, 10, 10)


(700, 10, 10)
for threshold (700, 10, 10) time: 0.0503807999775745
for threshold (10000, 1000, 100) time: 0.04987490002531558


# <span style="color:green"> Packaging 

Quand vous faites un pip install, vous installez un package. Ces packages sont des archives qui contiennent des modules Python et qui sont accessibles via le Python Package Index (PyPI) par exemple.
    
## Structure d'un package
Un package est un ensemble de modules qui peuvent être importés dans un programme Python. 

<pre>

packaging_tutorial/
├── LICENSE
├── pyproject.toml
├── README.md
├── src/
│   └── example_package/
│       ├── __init__.py
│       └── example.py
└── tests/
    └── test_example.py
    
</pre>

Le fichier __init__.py est un fichier spécial qui indique à Python que le répertoire contient un package. Il peut être vide ou contenir du code d'initialisation pour le package.

Le fichier example.py contient le code du package. Il peut contenir des classes, des fonctions, des variables, etc. Pour importer le module example.py, vous pouvez utiliser la commande suivante :

<pre>
from example_package import example
</pre>


## Outils de packaging

Les outils comme pip (build frontend ou frontend de construction) ne convertissent pas réellement vos sources en un paquet de distribution (comme un wheel). Ce travail est effectué par un backend de construction (build backend). Les backends de construction ont différents niveaux de fonctionnalité et vous devez en choisir un qui corresponde à vos besoins et à vos préférences. 

Le fichier pyproject.toml dira au frontend de construction quel backend de construction utiliser. Par exemple, pour utiliser le backend de construction de setuptools, vous pouvez ajouter la section suivante à votre fichier pyproject.toml :

<pre>
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"
</pre>

Le champs <code>requires</code> indique les dépendances nécessaires pour créer le package. Le champs <code>build-backend</code> indique le backend de construction à utiliser.

## Métadonnées

Les métadonnées sont des informations sur le package. Elles sont stockées dans le fichier <code>pyproject.toml</code>, par exemple :

```ini
[project]
name = "example-package"
version = "0.1.0"
description = "An example Python package"

author = "Your Name"

readme = "README.md"
requires-python = ">=3.8"
classifiers = [
    "Programming Language :: Python :: 3",
    "License :: OSI Approved :: MIT License",
    "Operating System :: OS Independent",
]

dependencies = [
    "numpy"
]

[project.urls]
Homepage = "https://github.com/pypa/sampleproject"
Issues = "https://github.com/pypa/sampleproject/issues"

```

## Génération du package 

Pour générer le package, vous pouvez utiliser la commande suivante :

<pre>
python -m build
</pre>

Cela générera un fichier wheel et un fichier tar.gz dans le répertoire dist.

## Installation du package

Pour installer le package, vous pouvez utiliser la commande suivante :

<pre>
pip install example-package-0.1.0-py3-none-any.whl
</pre>




# <span style="color:green"> Programmation fonctionnelle

La programmation fonctionnelle est un paradigme de programmation qui traite le calcul comme une évaluation de fonctions mathématiques et évite les changements d'état et les données mutables.

En Python, la programmation fonctionnelle est supportée par des fonctions intégrées comme map(), filter(), reduce(), etc. Ces fonctions prennent en entrée d'autres fonctions et des séquences pour effectuer des opérations sur les données.

In [12]:
# Exemple de programmation impérative ou procédurale

nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = 0
for num in nums:
    if num % 2 == 0:
        result += num * 10
print(result)

# Exemple de programmation fonctionnelle

result = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = filter(lambda x: x % 2 == 0, result)
result = map(lambda x: x * 10, result)
result = sum(result)
print(result)


300
300


PySpark est un exemple de framework qui utilise la programmation fonctionnelle pour traiter de grands ensembles de données. PySpark est l'API Python pour Apache Spark, un framework de calcul distribué. 


Généralement sur les workers, on fait des map, filter, reduce, etc. pour traiter les données. Ensuite, on fait des actions pour récupérer les résultats sur le driver (nœud principal) et c'est là où la réduction ou l'accumulation finale est effectuée.




# <span style="color:green"> Programmation sur GPU

Les GPUs (Graphics Processing Units) sont des processeurs spécialisés qui sont conçus pour effectuer des calculs sur des données massivement parallèles. Ils sont utilisés dans les jeux vidéo, la modélisation 3D, la cryptographie, le machine learning, etc.
<center>
<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2020/04/GPU-transistor.png" width="600"/>
</center>
Ils sont très souvent utilisés pour accélérer les calculs en machine learning. Par exemple, les bibliothèques comme TensorFlow et PyTorch utilisent les GPUs pour accélérer les calculs. Généralement, les librairies de machine learning utilisent CUDA, une API de NVIDIA, pour communiquer avec les GPUs. 
Pour ce qui suit, on va utiliser une version de PyTorch compatible CUDA pour tester (à utiliser sur Google colab avec le runtime GPU).

In [1]:
import torch

print(torch.__version__)

# test si un GPU compatible CUDA est disponible
# si vous avez un mac avec un Apple Silicon, vous pouvez utiliser le device mps
device = "cuda" if torch.cuda.is_available() else "cpu"

my_tensor = torch.tensor([[1,2,3],[4,5,6]], dtype=torch.float32)

print(my_tensor)

my_tensor = my_tensor.to(device)

print(my_tensor)

2.5.0+cu121
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0')


Maintenant pour montrer l'accélération, on va faire une opération simple de multiplication de matrices : 


In [16]:
import time

x = torch.randn(10000, 10000)
y = torch.randn(10000, 10000)
start = time.time()
z = x @ y
print(f"Time taken without cuda:  {time.time() - start:.5f}")

x = torch.randn(10000, 10000).cuda()
y = torch.randn(10000, 10000).cuda()
start = time.time()
z = x @ y
print(f"Time taken with cuda:  {time.time() - start:.5f}")



Time taken without cuda:  1.53525
Time taken with cuda:  0.00000


Les réseaux de neurones font beaucoup de calculs sur des matrices, donc l'utilisation des GPUs est très importante pour accélérer les calculs.

# Autodiff (Différentiation automatique)

Les réseaux de neurones utilisent la descente de gradient pour minimiser les fonctions de coût. Pour calculer les gradients des fonctions de coût par rapport aux paramètres du modèle, il faudrait spécifier les dérivées de ces fonctions. Cela peut être très difficile pour des fonctions complexes.

C'est ici que l'autodifférentiation entre en jeu. L'autodifférentiation est une technique qui permet de calculer les gradients automatiquement. PyTorch offre cette fonctionnalité avec son module autograd.

Dans la plupart des cas, vous n'avez pas besoin de vous soucier de la manière dont les gradients sont calculés. Vous pouvez simplement définir les opérations que vous souhaitez effectuer et PyTorch s'occupera du reste. Par exemple : 

In [2]:
import torch

# entrées du neurone
x = torch.tensor([4.0, 5.0]) 
# sortie attendue
y = torch.tensor(2.0) # sortie attendue

# un neurone avec un vecteur de poids (w) ainsi qu'un biais (b)
w = torch.tensor([1.0, 1.0], requires_grad=True)
b = torch.tensor(1.0, requires_grad=True)

# la sortie du neurone
y_hat = w * x + b

# la fonction de coût à optimiser
loss = torch.sum((y_hat - y)**2)

print("loss: ", loss)
print()

# calcul du gradient de la fonction de coût par rapport aux paramètres du modèle par autodiff
loss.backward()

print("gradient par autodiff de w: ", w.grad)
print("gradient par autodiff de b: ", b.grad)
print()

print("gradient théorique de w: ", 2 * x * ((w * x + b) - y))
print("gradient théorique de b: ", sum(2 * ((w * x + b) - y)))

# on peut mettre à jour les paramètres du modèle avec un optimiseur (SGD par exemple)
optimizer = torch.optim.SGD([w, b], lr=0.01)
optimizer.step()

print("nouveau w: ", w)
print("nouveau b: ", b)

print("nouvelle loss: ", (w * x + b - y).sum()**2)

# on voit que la loss a diminué 

loss:  tensor(25., grad_fn=<SumBackward0>)

gradient par autodiff de w:  tensor([24., 40.])
gradient par autodiff de b:  tensor(14.)

gradient théorique de w:  tensor([24., 40.], grad_fn=<MulBackward0>)
gradient théorique de b:  tensor(14., grad_fn=<AddBackward0>)
nouveau w:  tensor([0.7600, 0.6000], requires_grad=True)
nouveau b:  tensor(0.8600, requires_grad=True)
nouvelle loss:  tensor(14.1376, grad_fn=<PowBackward0>)


# <span style="color:green"> Pratique 
# Exercice 1

Créer un package python "calculatrice" qui prend en entrée deux vecteurs de même taille et affiche le résultat de l'opération sélectionnée (addition, soustraction, multiplication, division et le produit scalaire).
 
Une fois le package installé. L'utilisation doit se faire comme suit :

<pre>
python -m calculatrice --a 1 2 3 4 5 --b 6 7 8 9 10 --op addition
</pre>


Pour lire les arguments, vous pouvez utiliser le module argparse de Python. Je vous laisse faire de la documentation pour le package. Les arguments sont les suivants :

| Argument | Description | Type      |
|----------|-------------|-----------|
| --a      | vecteur a   | List[int] |
| --b      | vecteur b   | List[int] |
| --op     | opération   | str       |


# Exercice 2

Créer un réseau de neurones qu'on entrainera sur mnist avec PyTorch. Pour cela, vous pouvez utiliser le module torch.nn pour définir le réseau de neurones et le module torch.optim pour optimiser le réseau. Vous pouvez utiliser le dataset mnist de torchvision pour charger les données. 

- Le réseau doit avoir une architecture [100, 50, 10] avec des fonctions d'activation ReLU. 
- La fonction de coût doit être la "cross entropy".
- L'optimiseur doit être SGD avec un learning rate de 0.01.
- Vous pouvez utiliser deux DataLoader pour charger les données d'entrainement et de test.
- Pour l'entrainement, vous pouvez utiliser une boucle d'entrainement avec 10 itérations avec des batchs de 32.
- Une fois terminé, tester sur les données de test et afficher la précision du modèle. 
- À la fin, affichez 10 images de test avec la prédiction du modèle.